In [1]:
import autorootcwd
import torch
import torch.nn as nn
from torchvision import transforms
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from src.datasets.splits import get_ottawa2023_splits
from src.datasets.ottawa2023 import Ottawa2023Dataset
from src.datasets.dataset_wrapper import multiplydataset
from src.datasets.transforms import ToTensor, Normalize, FFT, RandomGain
from src.datasets.dataloaders import get_dataloaders
from src.models.dcnn import Net
from src.trainer.train import train
from src.trainer.test import test
import wandb


In [2]:
# --- Configs ---
SEED = 42
BATCH_SIZE = 64
EPOCHS = 20
FAULT_TYPE = "cage"
SAMPLE_LENGTH = 1.0 #in seconds
REPEATS = 10
LEARNING_RATE = 1e-3
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
wandb.login()

wandb: Currently logged in as: tth-bitt (tth-bitt-universidade-federal-de-santa-catarina) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(project="ottawa2023-fault-detection", 
           name=f"DCNN08_{FAULT_TYPE}_bs{BATCH_SIZE}_lr{LEARNING_RATE}_{EPOCHS}epochs",
           config={
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "learning_rate": LEARNING_RATE,
    "model": "DCNN08",
    "fault_type": FAULT_TYPE,
})

In [5]:
full_df = pd.read_pickle('/data/bearing_datasets/ottawa/processed/full_dataset.bz2')

In [6]:
#split
train_df, val_df, test_df = get_ottawa2023_splits(full_df)

In [7]:
# --- Define transforms ---
train_transform = transforms.Compose([
    ToTensor()])

val_test_transform = transforms.Compose([
    ToTensor()])

In [8]:
# --- Create datasets ---
train_dataset = Ottawa2023Dataset(train_df, faulty_type=FAULT_TYPE, sample_length=SAMPLE_LENGTH, transform=train_transform, random_sample=True)
val_dataset   = Ottawa2023Dataset(val_df, faulty_type=FAULT_TYPE, sample_length=SAMPLE_LENGTH, transform=val_test_transform, random_sample=False)
test_dataset  = Ottawa2023Dataset(test_df, faulty_type=FAULT_TYPE, sample_length=SAMPLE_LENGTH, transform=val_test_transform, random_sample=False)

In [9]:
# --- Multiply training set ---
train_dataset = multiplydataset(train_dataset, repeats=REPEATS)

In [10]:
# --- Create dataloaders ---
train_loader, val_loader, test_loader = get_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=BATCH_SIZE, num_workers=4)

In [11]:
# --- check device ---
print(f"Using device: {DEVICE}")

Using device: cuda


In [12]:
# --- Model, Loss, Optimizer ---
model = Net("DCNN08", in_channels=1, n_class=1).to(DEVICE)
loss_BCE = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [13]:
for epoch in range(EPOCHS):
    train_loss, auroc = train(dataloader = train_loader, model = model, loss_fn = loss_BCE, optimizer = optimizer, device = DEVICE, epoch=epoch)
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {train_loss:.4f}, AUROC: {auroc:.4f}")
    test(dataloader = val_loader, model = model, loss_fn = loss_BCE, device = DEVICE, epoch = epoch, val_or_test='val')
    test(dataloader = test_loader, model = model, loss_fn = loss_BCE, device = DEVICE, epoch = epoch, val_or_test='test')
    

Epoch 1/20, Loss: 0.3300, AUROC: 0.8710
[val] Epoch 0 - Loss: 0.2169, val_AUROC: 0.9991
[test] Epoch 0 - Loss: 0.1336, test_AUROC: 1.0000
Epoch 2/20, Loss: 0.1811, AUROC: 0.9654
[val] Epoch 1 - Loss: 0.2184, val_AUROC: 0.9966
[test] Epoch 1 - Loss: 0.0727, test_AUROC: 1.0000
Epoch 3/20, Loss: 0.1576, AUROC: 0.9688
[val] Epoch 2 - Loss: 0.0918, val_AUROC: 1.0000
[test] Epoch 2 - Loss: 0.0214, test_AUROC: 1.0000
Epoch 4/20, Loss: 0.1133, AUROC: 0.9851
[val] Epoch 3 - Loss: 0.3312, val_AUROC: 0.9841
[test] Epoch 3 - Loss: 0.3698, test_AUROC: 0.7864
Epoch 5/20, Loss: 0.0932, AUROC: 0.9904
[val] Epoch 4 - Loss: 0.0624, val_AUROC: 1.0000
[test] Epoch 4 - Loss: 0.0596, test_AUROC: 1.0000
Epoch 6/20, Loss: 0.0819, AUROC: 0.9900
[val] Epoch 5 - Loss: 0.0923, val_AUROC: 1.0000
[test] Epoch 5 - Loss: 0.0707, test_AUROC: 1.0000
Epoch 7/20, Loss: 0.0581, AUROC: 0.9972
[val] Epoch 6 - Loss: 0.0441, val_AUROC: 1.0000
[test] Epoch 6 - Loss: 0.0521, test_AUROC: 1.0000
Epoch 8/20, Loss: 0.0493, AUROC: 0